In [1]:
# Web Scraping Dependancies
import requests
from bs4 import BeautifulSoup

# Web Browsing Dependancies
import time
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

# Selenium Dependancies
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

# Pandas Dependancy
import pandas as pd

# Install Geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="MercuryOpenSM")

# Import the RateLimiter to enable us to do a large pull
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [2]:
# Test Geopy
address ="Athens,GA,us"
country ="us"
#loc = geolocator.geocode(city+','+ country)
loc = geolocator.geocode(address)
print(loc.address)
print("latitude is :" ,loc.latitude,"\nlongtitude is:" ,loc.longitude)

Athens-Clarke County Unified Government, Athens-Clarke County, Georgia, United States
latitude is : 33.9597677 
longtitude is: -83.376398


In [3]:
# Create a webdriver with Selenium
driver = webdriver.Chrome(executable_path="/Applications/chromedriver")
driver.get("https://www.linkedin.com/jobs/search/?keywords=Data%20Analyst&location=United%20States&locationId=&geoId=103644278&f_TPR=r86400&f_JT=F&f_E=2%2C3%2C4%2C5&f_WT=1&position=1&pageNum=0")

# Allow 2 seconds for the webpage to open
time.sleep(2)

/var/folders/8c/hshv2f6d619cmf35wk4k7f080000gn/T/ipykernel_54260/1206142728.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/Applications/chromedriver")


In [4]:
# Scroll through the page (and click "Show more jobs") 20 times
for x in range(20):
    
    # Get the screen height of the web page
    screen_height = driver.execute_script("return window.screen.height;")
    # Set the scroll pause time
    scroll_pause_time = 1
    # Initiate the index variable
    i = 1
    
    # Script for scrolling through the page
    while True:
        # Scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        
        # Update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;") 
        
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break
            
    # Identify the "Show more jobs" button
    element = driver.find_element(By.CLASS_NAME, 'infinite-scroller__show-more-button');

    # Click on the "Show more jobs" button if it is clickable
    try:
        time.sleep(scroll_pause_time)
        element.click()
    except:
        break

In [5]:
soup = BeautifulSoup(driver.page_source, "html.parser")
print(soup.prettify())

<html lang="en">
 <head>
  <meta content="d_jobs_guest_search" name="pageKey"/>
  <meta content="urlType=jserp_custom;emptyResult=false" name="linkedin:pageTag"/>
  <meta content="en_US" name="locale"/>
  <meta data-app-version="2.0.1117" data-browser-id="01c35a81-4ca9-4936-8e29-676d03c1f747" data-call-tree-id="AAXqYniQKhUJU+j/9VDVkg==" data-disable-jsbeacon-pagekey-suffix="false" data-enable-page-view-heartbeat-tracking="" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_search;nCs53nbIQx2c2h4XyZEJUA==" data-service-name="jobs-guest-frontend" id="config"/>
  <link href="https://www.linkedin.com/jobs/data-analyst-jobs" rel="canonical"/>
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <link href="https://static-exp1.licdn.com/sc/h/al2o9zrvru7aqj8e1x2rzsrca" rel="icon"/>
  <script async="" charset="utf-8" id="utag_94" src="https://platform.linkedin.com/litms/vendor/google/gtag-cm-dv360-sa360.js?id=DC-9261636" type="text

The job title is in an 'h3' tag with the following class: "base-search-card__title"
The job company is in an 'a' tag with the following class: "hidden-nested-link"
The job location is in a 'span' tag the following class: "job-search-card__location"

In [6]:
# Create a list of job titles from the LinkedIn Search
job_titles_list = []

# Scrape the h3 tags with the following class: "base-search-card__title"
raw_job_titles = soup.find_all('h3', class_='base-search-card__title')

# Add the text of the job title to the job titles list
for job_title in raw_job_titles:
    # Clean the job title to only include the text of the job title
    clean_job_title = job_title.text.replace('\n', '').strip()
    # Append the clean job title to the list
    job_titles_list.append(clean_job_title)

len(job_titles_list)

156

In [7]:
# Create a list of job companies from the LinkedIn Search
job_companies_list = []

# Scrape the a tags with the following class: "hidden-nested-link"
raw_job_companies = soup.find_all('a', class_='hidden-nested-link')

# Add the text of the job company to the job companies list
for job_company in raw_job_companies:
    clean_job_company = job_company.text.replace('\n','').strip()
    job_companies_list.append(clean_job_company)
    
len(raw_job_companies)
    
# len(job_companies_list)

156

In [8]:
# Create a list of job locations from the LinkedIn Search
job_locations_list = []

# Scrape the h3 tags with the following class: "base-search-card__title"
raw_job_locations = soup.find_all('span', class_='job-search-card__location')

# Add the text of the job location to the job locations list
for job_location in raw_job_locations:
    clean_job_location = job_location.text.replace('\n', '').strip()
    job_locations_list.append(clean_job_location)
    
len(job_locations_list)

156

In [9]:
# Create a list of dictionaries for jobs from LinkedIn
linkedin_jobs = []

# Loop through the job lists to create and append the Linkedin Job Dictionaries
for i in range(len(job_titles_list)):
    linkedin_job_dictionary = {}
    linkedin_job_dictionary['job_title'] = job_titles_list[i]
    #linkedin_job_dictionary['job_company'] = job_companies_list[i]
    linkedin_job_dictionary['job_location'] = job_locations_list[i]
    
    # Append the job description if:
    # There is a comma in the location (City, State), and 'United States' is not in the location
    if (',' in linkedin_job_dictionary['job_location']) and not \
    ('United States' in linkedin_job_dictionary['job_location']):
        linkedin_jobs.append(linkedin_job_dictionary)

len(linkedin_jobs)

154

In [10]:
def grabCoordinates(job):
    loc = job['job_location']
    split_string = loc.split(",", 1)
    city = split_string[0]
    state = split_string[1].strip()

    # Append the usable address to each of the job dictionaries
    cityStateCountry = city + "," + state + ",us"

    # Search for Nominatim the lat and lon
    time.sleep(2)
    nominatimLoc = geolocator.geocode(cityStateCountry)
    job['latitude'] = nominatimLoc.latitude
    job['longitude'] = nominatimLoc.longitude

In [11]:
# Loop through the job dictionaries    
for i in range(3):
    grabCoordinates(linkedin_jobs[i])

In [12]:
# Loop through the job dictionaries
for job in linkedin_jobs:
    grabCoordinates(job)

In [13]:
linkedin_jobs

[{'job_title': 'Data Analyst',
  'job_location': 'Austin, TX',
  'latitude': 30.2711286,
  'longitude': -97.7436995},
 {'job_title': 'Data Analyst, Technical Operations, TDaaS',
  'job_location': 'Atlanta, GA',
  'latitude': 33.7489924,
  'longitude': -84.3902644},
 {'job_title': 'Data Analyst',
  'job_location': 'New York, NY',
  'latitude': 40.7127281,
  'longitude': -74.0060152},
 {'job_title': 'Data Analyst/IT',
  'job_location': 'St Louis, MO',
  'latitude': 38.6319657,
  'longitude': -90.2428756},
 {'job_title': 'Data Analyst',
  'job_location': 'Huntsville, AL',
  'latitude': 34.729847,
  'longitude': -86.5859011},
 {'job_title': 'Data Analyst',
  'job_location': 'Elizabeth, NJ',
  'latitude': 40.6639916,
  'longitude': -74.2107006},
 {'job_title': 'Senior Data Analyst, Marketing',
  'job_location': 'Washington, DC',
  'latitude': 38.8950368,
  'longitude': -77.0365427},
 {'job_title': 'Entry Level Data Analyst (340191)',
  'job_location': 'Dallas, TX',
  'latitude': 32.7762719,

In [14]:
linkedinDF = pd.DataFrame(linkedin_jobs)
linkedinDF

,job_title,job_location,latitude,longitude
0,Data Analyst,"Austin, TX",30.271129,-97.743700
1,"Data Analyst, Technical Operations, TDaaS","Atlanta, GA",33.748992,-84.390264
2,Data Analyst,"New York, NY",40.712728,-74.006015
3,Data Analyst/IT,"St Louis, MO",38.631966,-90.242876
4,Data Analyst,"Huntsville, AL",34.729847,-86.585901
...,...,...,...,...
149,Supply Chain Business Analyst / Data Analyst,"Portsmouth, VA",36.844420,-76.353300
150,HRIS Business Intelligence Analyst,"Philadelphia, PA",39.952724,-75.163526
151,Sr. Data and Analytics Business Analyst,"Wayzata, MN",44.970759,-93.511947
152,Data Science Analyst,"San Antonio, TX",29.424600,-98.495141


In [17]:
linkedinDF.to_csv('linkedin.csv', index=False)

In [38]:
driver.quit()